## Web Scraping Zillow.com

In [ ]:
# Importing all libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import json

In [ ]:
# initiating user agent to avoid getting blocked
header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36",
  'referer': 'https://www.zillow.com/homes/for_rent/Manhattan,-New-York,-NY_rb'

}
# passing our url
url = 'https://www.zillow.com/homes/for_rent/Manhattan,-New-York,-NY_rb'

In [ ]:
r = requests.get(url=url,headers=header)
r

<Response [200]>

In [ ]:
# making the soup object
soup= BeautifulSoup(r.text, 'lxml')
# lets look for the next page tag so we can extract urls for number of pages we want to scrape

url+soup.find('a', rel='next')['href']

'https://www.zillow.com/homes/for_rent/Manhattan,-New-York,-NY_rb/manhattan-new-york-ny/rentals/2_p/'

In [ ]:
# getting the url of 10 pages we want to scrape
# we'll extract data for the first 10 pages
urls = []
for i in range(1,11):
    a = 'https://www.zillow.com/homes/for_rent/Manhattan,-New-York,-NY_rb/manhattan-new-york-ny/rentals/'+str(i)+'_p/'
    urls.append(a)

In [ ]:
import requests
import re
import json

req_headers = header # Headers to be used in the requests
data_list = []

with requests.Session() as s:
    for url in urls:
        response = s.get(url, headers=req_headers)
        html_content = response.text

        # Extract essential data using regex
        data = json.loads(re.search(r'!--(\{"queryState".*?)-->', html_content).group(1))

        # Append extracted data to the data_list
        data_list.append(data)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

df = pd.DataFrame()

def make_data_frame(frame):
    for i in data_list:
        for item in i['cat1']['searchResults']['listResults']: # looping the dictionary of essential data created earlier
            frame = frame.append(item, ignore_index=True)
    return frame

df = make_data_frame(df)

In [ ]:
df.head()

,zpid,id,lotId,imgSrc,hasImage,carouselPhotos,statusType,statusText,detailUrl,latLong,...,hasVideo,isHomeRec,hasAdditionalAttributions,availabilityDate,hasOpenHouse,openHouseStartDate,openHouseEndDate,openHouseDescription,streetViewMetadataURL,streetViewURL
0,40.73736_rrc2*-rrc_73.97319,40.73736_rrc2*-rrc_73.97319,2.008579e+09,https://photos.zillowstatic.com/fp/c5384b7a159...,True,[{'url': 'https://photos.zillowstatic.com/fp/c...,FOR_RENT,Waterside,/apartments/new-york-ny/waterside/97QdBH/,"{'latitude': 40.73736, 'longitude': -73.97319}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40.704678_rrc2*-rrc_74.007576,40.704678_rrc2*-rrc_74.007576,1.000001e+09,https://photos.zillowstatic.com/fp/39db1832519...,True,[{'url': 'https://photos.zillowstatic.com/fp/3...,FOR_RENT,95 Wall,/apartments/new-york-ny/95-wall/5XSsMP/,"{'latitude': 40.704678, 'longitude': -74.007576}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,40.751972_rrc2*-rrc_73.9857,40.751972_rrc2*-rrc_73.9857,1.390700e+04,https://photos.zillowstatic.com/fp/c71bbf21380...,True,[{'url': 'https://photos.zillowstatic.com/fp/c...,FOR_RENT,Atlas New York,/apartments/manhattan-ny/atlas-new-york/27nq/,"{'latitude': 40.751972, 'longitude': -73.9857}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,40.711334_rrc2*-rrc_74.0168,40.711334_rrc2*-rrc_74.0168,1.001493e+09,https://photos.zillowstatic.com/fp/f669ca5fae3...,True,[{'url': 'https://photos.zillowstatic.com/fp/f...,FOR_RENT,Gateway,/apartments/new-york-ny/gateway/5XjY8c/,"{'latitude': 40.711334, 'longitude': -74.0168}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,40.76221_rrc2*-rrc_73.99658,40.76221_rrc2*-rrc_73.99658,1.001684e+09,https://photos.zillowstatic.com/fp/e1d85d1a740...,True,[{'url': 'https://photos.zillowstatic.com/fp/e...,FOR_RENT,Gotham West,/apartments/manhattan-ny/gotham-west/5XmBQy/,"{'latitude': 40.76221, 'longitude': -73.99658}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.to_csv('zillow_data.csv', index=False)